## Optimization Attempt 1
In an attempt to optimize the rf model, features with low PCA loadings will be dropped in this trial (sex, BPMeds, prevalentStroke, heartRate)

In [19]:
# Importing dependencies
import pandas as pd
from pathlib import Path
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline


In [20]:
# Reading in the data
file_path = Path('../Resources/cleaned_data_map.csv')
df = pd.read_csv(file_path)
df.head()

,sex,age,education,smokingStatus,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,BMI,heartRate,glucose,CHDRisk,MAP
0,1.0,39,4,0.0,0,0,0,0,0,195,26.97,80,77,0,82.000000
1,0.0,46,2,0.0,0,0,0,0,0,250,28.73,95,76,0,94.333333
2,1.0,48,1,1.0,20,0,0,0,0,245,25.34,75,70,0,95.833333
3,0.0,61,3,1.0,30,0,0,1,0,225,28.58,65,103,1,113.333333
4,0.0,46,3,1.0,23,0,0,0,0,285,23.10,85,85,0,99.333333


In [21]:
df.drop(columns=['sex', 'BPMeds', 'prevalentStroke', 'heartRate'], inplace=True)
df.head()

,age,education,smokingStatus,cigsPerDay,prevalentHyp,diabetes,totChol,BMI,glucose,CHDRisk,MAP
0,39,4,0.0,0,0,0,195,26.97,77,0,82.000000
1,46,2,0.0,0,0,0,250,28.73,76,0,94.333333
2,48,1,1.0,20,0,0,245,25.34,70,0,95.833333
3,61,3,1.0,30,1,0,225,28.58,103,1,113.333333
4,46,3,1.0,23,0,0,285,23.10,85,0,99.333333


In [22]:
# Separating target variables and features
y = df['CHDRisk']
X = df.drop(columns='CHDRisk')

In [23]:
y.value_counts()

CHDRisk
0    3084
1     553
Name: count, dtype: int64

In [24]:
# Splitting the data into testing and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [25]:
y_train.value_counts()

CHDRisk
0    2312
1     415
Name: count, dtype: int64

In [26]:
# Using synthetic minority over-sampling technique to balance the target variable conditions in the training data
smote = SMOTE(random_state=1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [27]:
y_train_resampled.value_counts()

CHDRisk
0    2312
1    2312
Name: count, dtype: int64

In [28]:
X_train_resampled.shape

(4624, 10)

In [29]:
y_train_resampled.shape

(4624,)

In [30]:
# Scaling the feature variables
scaler = StandardScaler()
X_scaler = scaler.fit(X_train_resampled)

X_train_scaled = X_scaler.transform(X_train_resampled)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
# Instantiating the model
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [32]:
# Training the model
rf_model = rf_model.fit(X_train_scaled, y_train_resampled)

In [33]:
# Making predictions with the testing data
test_predictions = rf_model.predict(X_test_scaled)

In [34]:
# Creating the confusion matrix
cm = confusion_matrix(y_test, test_predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, test_predictions)

## Testing Data Results

In [35]:
# Creating the confusion matrix
cm = confusion_matrix(y_test, test_predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy and recall scores
acc_score = accuracy_score(y_test, test_predictions)

In [36]:
# Printing the results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, test_predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,646,126
Actual 1,101,37


Accuracy Score : 0.7505494505494505
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       772
           1       0.23      0.27      0.25       138

    accuracy                           0.75       910
   macro avg       0.55      0.55      0.55       910
weighted avg       0.77      0.75      0.76       910



## Conclusion
Removing the features with low PCA loadings (sex, BPMeds, prevalentStroke, heartRate) did not improve accuracy and overall model performance.